<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Erin/check_European_Call_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nstock = 1

# Test (Skip this if not trying to test, to make sure that functions are defined correctly in cells below without running this cell)

In [2]:
# import jax
# import jax.numpy as jnp
# from jax import random
# from jax import jit
# import numpy as np
# from torch.utils.dlpack import from_dlpack

# def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
#     stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
#     stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
#                             jax.ops.index[0],         # initialization of stock prices
#                             initial_stocks)
#     noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
#     sigma = jnp.diag(cov) ** (1/2)
#     dt = T / numsteps
#     def time_step(t, val):
#         dx = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
#         val = jax.ops.index_update(val,
#                             jax.ops.index[t],
#                             val[t-1] * dx)
#         return val
#     return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

# def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T):
#   return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-K,0)) * jnp.exp(-r[0] * T))

# numstocks = nstock
# numsteps = 50
# numpaths = 100000

# rng = jax.random.PRNGKey(np.random.randint(10000))
# rng, key = jax.random.split(rng)

# drift = jnp.array([0.]*numstocks)
# r = drift
# cov = jnp.identity(numstocks)*0.25*0.25
# initial_stocks = jnp.array([100.]*numstocks) # must be float
# T = 1.0
# K = 110.0 

# fast_simple = jax.jit(Brownian_motion, static_argnums=2)
# keys = jax.random.split(key, numpaths)
# batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# # option price
# print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T))
# %timeit optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T)

# # delta test
# goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T))
# %timeit goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T)

# Construct Neural Net

In [3]:
%%writefile cupy_dataset.py
import cupy
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        dx = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, keys): # need to pass 'keys'
    return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-K,0)) * jnp.exp(-r[0] * T))

###################################################################################################
# these 2 functions must be defined outside class in order to be used in 'optionvalueavg' function
fast_simple = jax.jit(Brownian_motion, static_argnums=2)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))
###################################################################################################

class OptionDataSet(object):
    
    def __init__(self, max_len, number_path, batch, seed, stocks):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 50
        self.N_BATCH = batch
        self.N_STOCKS = stocks
        self.T = 1.0 # assume T = 1, use float here
        self.seed = seed
        np.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num >= self.max_length:
            raise StopIteration
        
        # Y = cupy.zeros((self.N_BATCH, 1 + self.N_STOCKS), dtype=cupy.float32) # output: price, delta1, delta2, delta3
        Y = cupy.zeros((self.N_BATCH), dtype = cupy.float32)
        X = cupy.zeros((self.N_BATCH, self.N_STOCKS * 6), dtype = cupy.float32)

        for op in range(self.N_BATCH):
          
          rng = jax.random.PRNGKey(self.seed)
          rng, key = jax.random.split(rng)

          ################################################################################################### generate random input numbers

          initial_stocks = jnp.array(np.random.random(self.N_STOCKS) * 1.0)

          corr = jnp.diag(jnp.array([1]*self.N_STOCKS)) # assume no correlation between stocks here
          sigma = jnp.array(np.random.random(self.N_STOCKS) * 0.3)
          cov = (jnp.diag(sigma)).dot(corr).dot(jnp.diag(sigma))

          r = jnp.repeat(jnp.array(np.random.random(1) * 0.1), self.N_STOCKS)
          drift = jnp.array(np.random.random(self.N_STOCKS) * 0.1)

          T = self.T
          K = np.random.random(1) * 1.0

          ###################################################################################################
          ################################################################################################### apply functions to compute price and deltas
          
          keys = jax.random.split(key, self.N_PATHS)

          # European_Call_price = optionvalueavg(key, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys) # need to pass 'keys'
          # gooptionvalue = jax.grad(optionvalueavg, argnums=1)
          # Deltas = gooptionvalue(keys, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys) # need to pass 'keys'

          ###################################################################################################
          ################################################################################################### store input and output numbers in X and Y

          # Y[op, 0] = European_Call_price
          # Y[op, 1:4] = cupy.array(Deltas, dtype=cupy.float32)
          Y[op] = optionvalueavg(key, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys)

          # T, K, S, sigma, mu, r
          paras = (jnp.repeat(jnp.array(T), self.N_STOCKS), jnp.repeat(jnp.array(K), self.N_STOCKS), initial_stocks, sigma, drift, r)
          paras = np.column_stack(paras).reshape(1,-1)[0]
          X[op,] = cupy.array(paras)

          ###################################################################################################

        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))


# ds = OptionDataSet(max_len = 2, number_path = 10000, batch = 2, seed = 15, stocks=3) # for testing purpose, use constant seed. When training, change to random seed
# for i in ds:
#     print(i)

Overwriting cupy_dataset.py


In [4]:
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

class Net(nn.Module):

    def __init__(self, hidden=1024, nstock = 1):
        super(Net, self).__init__()
        self.nstock = nstock
        self.fc1 = nn.Linear(6*self.nstock, hidden) # remember to change this!
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 1) # 4 outputs: price, delta1, delta2, delta3
        self.register_buffer('norm',
                             torch.tensor([1, 1.0, 1.0, 0.3, 0.1, 0.1] * self.nstock)) # don't use numpy here - will give error later
                                                                               # T, K, S, sigma, mu, r

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Overwriting model.py


# Train Neural Net

In [5]:
!pip install pytorch-ignite

In [ ]:
# If memory is not enough, try changing parameters and restarting session
# loss will converge

from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import OptionDataSet
import numpy as np
import torch
from torch import Tensor
from torch.autograd import grad
timer = Timer(average=True)
model = Net(nstock = nstock).cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
dataset = OptionDataSet(max_len = 100, number_path = 1024, batch = 32, seed = np.random.randint(10000), stocks = nstock) # must have random seed

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    # print(x)
    y = batch[1]
    # print(y)
    y_pred = model(x)
    # print(y_pred)

    # def compute_deltas(x):
    #   inputs = x
    #   inputs.requires_grad = True
    #   first_order_gradient = grad(model(inputs), inputs, create_graph=False)
    #   return first_order_gradient[0][[2,8,14]]

    # # print(torch.unbind(x))
    # # print([compute_deltas(x) for x in torch.unbind(x)])
    # # print(torch.stack([compute_deltas(x) for x in torch.unbind(x)], dim=0))

    # deltas = torch.stack([compute_deltas(x) for x in torch.unbind(x)], dim=0)
    # y_pred = torch.cat((y_pred, deltas), 1)
    # # print(y_pred)

    # loss_weight = 1/(y.mean(axis=0)**2)
    # # print(y.mean(axis=0))
    # # print((y.mean(axis=0)**2))
    # # print(1/(y.mean(axis=0)**2))
    
    # loss_weight_normalized = loss_weight/loss_weight.sum()
    # # print(loss_weight)
    # # print(loss_weight_normalized)
    # loss = ((y_pred - y) ** 2 * loss_weight_normalized).mean() # compute weighted MSE between the 2 arrays
    # # print((y_pred - y) ** 2)
    # # print((y_pred - y) ** 2 * loss_weight_normalized)
    # # print(loss)

    loss = loss_fn(y_pred[:,0], y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 20

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output * 10000, 'average time', timer.value(), 'iter num', iter) # print by multiplying 10000 -> easier to read (actual loss function isn't amplified)
        
trainer.run(dataset, max_epochs = 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: /usr/local/lib/python3.7/dist-packages/ignite/contrib/handlers/param_scheduler.py has been moved to /ignite/handlers/param_scheduler.py and will be removed in version 0.6.0.
 Please refer to the documentation for more details.
  


loss 170.91235145926476 average time 0.316498195749989 iter num 20
loss 104.18352670967579 average time 0.1598302217249909 iter num 40
loss 127.41565704345703 average time 0.10753424153332768 iter num 60
loss 101.56030766665936 average time 0.08140095707499313 iter num 80
loss 107.39786550402641 average time 0.06571229763999555 iter num 100
loss 172.94719815254211 average time 0.027531507750040873 iter num 20


**Save Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
model_save_name = 'european_call_1_v2.pth'
path = F"/content/drive/MyDrive/AFP/Save_Models/{model_save_name}" 
torch.save(model.state_dict(), path)

**Load Model**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import torch
# model_save_name = 'european_call_1.pth'
# path = F"/content/drive/MyDrive/AFP/Save_Models/{model_save_name}" 
# state_dict = torch.load(path)
# print(state_dict.keys())

In [ ]:
# need to run 'Writing cupy_dataset.py' and 'Writing model.py' above before this
from model import Net
model = Net(nstock = nstock).cuda()

model.load_state_dict(state_dict)
print(model)

**Continue to train model**

In [ ]:
# # If memory is not enough, try changing parameters and restarting session
# # loss will converge

# from ignite.engine import Engine, Events
# from ignite.handlers import Timer
# from torch.nn import MSELoss
# from torch.optim import Adam
# from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
# from ignite.handlers import ModelCheckpoint
# from model import Net
# from cupy_dataset import OptionDataSet
# import numpy as np
# timer = Timer(average=True)
# #model = Net().cuda()
# loss_fn = MSELoss()
# optimizer = Adam(model.parameters(), lr=1e-3)
# #dataset = OptionDataSet(max_len = 100, number_path = 1024, batch = 32, seed = np.random.randint(10000), stocks = 3) # must have random seed
# dataset = OptionDataSet(max_len = 100, number_path = 100000, batch = 8, seed = np.random.randint(10000), stocks = 3) # must have random seed


# def train_update(engine, batch):
#     model.train()
#     optimizer.zero_grad()
#     x = batch[0]
#     # print(x)
#     y = batch[1]
#     # print(y)
#     y_pred = model(x)
#     # print(y_pred)

#     def compute_deltas(x):
#       inputs = x
#       inputs.requires_grad = True
#       first_order_gradient = grad(model(inputs), inputs, create_graph=False)
#       return first_order_gradient[0][[2,8,14]]

#     # print(torch.unbind(x))
#     # print([compute_deltas(x) for x in torch.unbind(x)])
#     # print(torch.stack([compute_deltas(x) for x in torch.unbind(x)], dim=0))

#     deltas = torch.stack([compute_deltas(x) for x in torch.unbind(x)], dim=0)
#     y_pred = torch.cat((y_pred, deltas), 1)
#     # print(y_pred)

#     loss_weight = 1/(y.mean(axis=0)**2)
#     # print(y.mean(axis=0))
#     # print((y.mean(axis=0)**2))
#     # print(1/(y.mean(axis=0)**2))
    
#     loss_weight_normalized = loss_weight/loss_weight.sum()
#     # print(loss_weight)
#     # print(loss_weight_normalized)
#     loss = ((y_pred - y) ** 2 * loss_weight_normalized).mean() # compute weighted MSE between the 2 arrays
#     # print((y_pred - y) ** 2)
#     # print((y_pred - y) ** 2 * loss_weight_normalized)
#     # print(loss)
#     loss.backward()
#     optimizer.step()
#     return loss.item()

# trainer = Engine(train_update)
# log_interval = 10

# scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
# trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
# timer.attach(trainer,
#              start=Events.EPOCH_STARTED,
#              resume=Events.ITERATION_STARTED,
#              pause=Events.ITERATION_COMPLETED,
#              step=Events.ITERATION_COMPLETED)    
# @trainer.on(Events.ITERATION_COMPLETED)
# def log_training_loss(engine):
#     iter = (engine.state.iteration - 1) % len(dataset) + 1
#     if iter % log_interval == 0:
#         print('loss', engine.state.output * 10000, 'average time', timer.value(), 'iter num', iter) # print by multiplying 10000 -> easier to read (actual loss function isn't amplified)
        
# trainer.run(dataset, max_epochs = 50)

# model_save_name = 'jax_european_test_3.pth'
# path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
# torch.save(model.state_dict(), path)

#Results

In [ ]:
import torch
inputs = torch.tensor([[1, 0.8, 0.8, 0.25, 0.05, 0.05]*nstock]).cuda() # T, K, S, sigma, mu, r
print('price: ' + str(model(inputs.float())))

inputs.requires_grad = True
x = model(inputs.float())
x.backward()
first_order_gradient = inputs.grad
first_order_gradient[0][[2,8,14]]

# price, delta1, delta2, delta3
# should be around (0.067710705, 0.22125466 , 0.22136934 , 0.22104672)

In [ ]:
numstocks = 1
numsteps = 50
numpaths = 100000

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)

drift = jnp.array([0.05]*numstocks)
r = drift
cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([0.8]*numstocks) # must be float
T = 1.0
K = 0.8

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T))

# Delta

In [ ]:
##Using gradient, Change only 1 S0 at a time
%matplotlib inline
from torch.autograd import grad
import pylab
import numpy as np
def compute_delta(S):
    inputs = torch.tensor([[1, 0.8, S, 0.25, 0.05, 0.05] + ([1, 0.8, 0.8, 0.25, 0.05, 0.05]*2)]).cuda()
    inputs.requires_grad = True
    x = model(inputs.float())
    x.backward()
    first_order_gradient = inputs.grad
    return first_order_gradient[0][2]
prices = np.arange(0, 1, 0.01)
deltas = []
for p in prices:
    deltas.append(compute_delta(p).item())
fig = pylab.plot(prices, deltas)
pylab.xlabel('prices')
pylab.ylabel('Delta')
fig